BUILDING A CHATBOT

Conversation RagChatbot : Enable a Chatbot experience over an external source of data 
Agents : Build a Chatbot that takes action

In [102]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [103]:
from langchain_groq import ChatGroq
# gemma2-9b-it was decommissioned; use a supported model (see https://console.groq.com/docs/models)
model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001D48699F160>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D48699DCF0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [104]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My Name is Mitarth Pandey and i am aspiring to become software engineering")])

AIMessage(content="Nice to meet you, Mitarth Pandey! It's great to hear that you're aspiring to become a software engineer. That's a fantastic career goal, and with dedication and hard work, you can achieve it.\n\nAs a software engineer, you'll have the opportunity to work on a wide range of technologies, from mobile apps to web applications, and from artificial intelligence to data science. You'll have the chance to solve complex problems, collaborate with a team, and continuously learn new skills.\n\nTo help you on your journey, I'd like to ask a few questions:\n\n1. What programming languages are you interested in learning?\n2. Do you have any experience with coding or software development?\n3. What areas of software engineering interest you the most (e.g., web development, mobile app development, data science, etc.)?\n4. Are you planning to pursue a degree in computer science or software engineering, or do you want to learn through online courses and bootcamps?\n\nLet me know your 

In [105]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi,My Name is Mitarth Pandey and i am aspiring to become software engineering"),
        AIMessage(content="Nice to meet you, Mitarth Pandey. It\'s great to hear that you\'re aspiring to become a software engineer. That\'s a fantastic career goal, and with dedication and hard work, you can achieve it.\n\nAs a software engineer, you\'ll have the opportunity to work on a wide range of projects, from developing mobile apps to creating complex enterprise software systems. You\'ll need to have strong programming skills, problem-solving abilities, and a passion for learning new technologies.\n\nTo help you get started, here are some general tips:\n\n1. **Learn the fundamentals**: Start by learning the basics of programming, including data structures, algorithms, and object-oriented programming.\n2. **Choose a programming language**: Select a language you enjoy working with, such as Python, Java, or JavaScript.\n3. **Practice, practice, practice**: Build small projects and contribute to open-source projects to gain practical experience.\n4. **Stay up-to-date with industry trends**: Follow industry leaders, blogs, and news outlets to stay current with the latest technologies and advancements.\n5. **Network with other engineers**: Join online communities, attend coding meetups, and connect with other engineers to learn from their experiences."),
        HumanMessage(content="What is my name what is it I do ?")
    ]
)

AIMessage(content='Your name is Mitarth Pandey.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 311, 'total_tokens': 320, 'completion_time': 0.013547868, 'completion_tokens_details': None, 'prompt_time': 0.017740991, 'prompt_tokens_details': None, 'queue_time': 0.045004429, 'total_time': 0.031288859}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c4cfb-b78d-71c3-8e20-9fd45bca987d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 311, 'output_tokens': 9, 'total_tokens': 320})

### MESSAGE HISTORY

We can use a message history class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore
Future Interactions will then load these messages and pass them into the chain as part of the input. Lets see how to do this 

In [106]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [107]:
config = {"configurable" : {"session_id" : "chat1"}}

In [108]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi,My Name is Mitarth Pandey and i am aspiring to become software engineering")],
    config=config
)

In [109]:
response.content

"Hello Mitarth Pandey,\n\nIt's great to meet you. Becoming a software engineer can be a rewarding and challenging career path. With the increasing demand for technology and digital solutions, the field of software engineering has a lot of opportunities.\n\nTo support your aspiration, I'd like to know more about your background and interests. Here are a few questions to get started:\n\n1. What programming languages are you familiar with, and which ones do you want to learn?\n2. What areas of software engineering interest you the most, such as web development, mobile app development, artificial intelligence, or data science?\n3. Do you have any prior experience in software development, or are you starting from scratch?\n4. What kind of projects or activities have you done in the past that you enjoyed or found challenging?\n5. Are you planning to pursue a degree in computer science or software engineering, or do you want to learn through online resources and self-study?\n\nFeel free to sh

In [110]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content='Your name is Mitarth Pandey.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 290, 'total_tokens': 299, 'completion_time': 0.009726433, 'completion_tokens_details': None, 'prompt_time': 0.020090439, 'prompt_tokens_details': None, 'queue_time': 0.046249161, 'total_time': 0.029816872}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c4cfb-ba2d-7d51-bd77-94b4a49dfe1b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 290, 'output_tokens': 9, 'total_tokens': 299})

In [111]:
## Change the config --> sessionid
config1={"configurable" : {"session_id" : "chat2"}}
with_message_history.invoke(
    [HumanMessage(content="What is my name ? ")],
    config = config1
)

AIMessage(content="I don't have any information about your name. I'm a large language model, I don't have the ability to retain information about individual users, and our conversation has just started, so I don't have any context about who you are. If you'd like to share your name, I'm happy to chat with you!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 41, 'total_tokens': 108, 'completion_time': 0.081120799, 'completion_tokens_details': None, 'prompt_time': 0.003114603, 'prompt_tokens_details': None, 'queue_time': 0.046388416, 'total_time': 0.084235402}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c4cfb-bac1-75b1-8bb4-3b015e7e4e25-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 41, 'output_tokens': 67, 'total_tokens': 108})

In [112]:
## Change the config --> sessionid
config1={"configurable" : {"session_id" : "chat2"}}
with_message_history.invoke(
    [HumanMessage(content="Hey my name is Mitarth Pandey  ")],
    config = config1
)

AIMessage(content="Nice to meet you, Mitarth Pandey. It's great that you're willing to share your name with me. I'll remember it for our conversation, but as I mentioned earlier, I don't retain information about individual users, so don't worry about me knowing it outside of this chat. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 126, 'total_tokens': 195, 'completion_time': 0.097144578, 'completion_tokens_details': None, 'prompt_time': 0.010905087, 'prompt_tokens_details': None, 'queue_time': 0.047028453, 'total_time': 0.108049665}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c4cfb-bb8f-7d91-883f-136a564c8d24-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 126, 'output_tokens': 69, 'total_tokens': 195})

## PROMPT TEMPLATES 


Prompt templates help to turn raw user information into a format that LLM can work with . in this case the raw user input is just a message, which we are passing to LLM
.Lets Now that a bit more complicated First Let add in a system message with some custom instructions(but still taking message as input ). Next we will add in more input besides just the messages 

In [113]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant that helps to answer questions best to your ability"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [114]:
chain.invoke({"messages" : [HumanMessage(content="Hi my name is Mitarth")]})


AIMessage(content="Nice to meet you, Mitarth. I'm happy to help you with any questions or topics you'd like to discuss. How's your day going so far? Is there something specific on your mind that you'd like to talk about, or would you like me to suggest some conversation starters?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 55, 'total_tokens': 115, 'completion_time': 0.080220442, 'completion_tokens_details': None, 'prompt_time': 0.002670112, 'prompt_tokens_details': None, 'queue_time': 0.045226937, 'total_time': 0.082890554}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c4cfb-bc7b-7403-957f-de176125ff76-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 55, 'output_tokens': 60, 'total_tokens': 115})

In [115]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [116]:
config = {"configurable" : {"session_id" : "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config,
)

response.content

"I'm happy to help, but I don't have any information about your personal identity. I'm a conversational AI assistant, and our conversation just started, so I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you. Alternatively, I can generate a random name if you'd like a fun and fictional identity. Just let me know!"

In [117]:
## Add more complexity 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the questions to best of your ability in {language}"
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model

In [118]:
response = chain.invoke({"messages" : [HumanMessage(content="Hi My name is Mitarth")], "language" : "Hindi"})
response.content

'नमस्ते Mitarth, मैं आपकी सहायता करने के लिए यहां हूं। क्या मैं आपकी किसी समस्या का समाधान करने में आपकी मदद कर सकता हूं?'

Lets now wrap this more complicated chain in a Message History class. this time , because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [119]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
    history_messages_key="messages"
)

In [120]:
# Set up the config for this conversation session
config = {"configurable" : {"session_id" : "chat4"}}

# Store information in the conversation
response = with_message_history.invoke(
    {'messages' :[HumanMessage(content="my name is Mitarth Pandey")],"language" : "Hindi"},
    config=config,
)

response.content   

'मैं आपकी सहायता करने के लिए तैयार हूँ। कृपया कोई प्रश्न पूछें, मैं आपको संभव से अधिक जानकारी प्रदान करने का प्रयास करूँगा।'

In [121]:
# Ask a question - the chatbot will remember previous messages from this session
message = with_message_history.invoke(
    {'messages' :[HumanMessage(content="What's my name?")],"language" : "Hindi"},
    config=config,
)
message.content

''

## MANAGE THE CONVERSATION

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded 
and potentially overflow the context window of the LLM. Therefore it is important to add a step limits the size of message you are passing in

In [126]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter=model,
    include_system=True,
    allow_partial = False,
    start_on="human"
)

messages = [
    SystemMessage(content="Hi I am a good assistant"),
    HumanMessage(content="Hi I'm Bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="What is 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]

trimmer.invoke(messages)

c:\Users\87504\Desktop\RAGLearning\.venv\lib\site-packages\langchain_core\language_models\base.py:336: UserWarning: Using fallback GPT-2 tokenizer for token counting. Token counts may be inaccurate for non-GPT-2 models. For accurate counts, use a model-specific method if available.
  return len(self.get_token_ids(text))
c:\Users\87504\Desktop\RAGLearning\.venv\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\87504\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Pyt

[SystemMessage(content='Hi I am a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Hi I'm Bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='What is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs=

In [131]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(message=itemgetter("messages") | trimmer) | prompt | model
)
response = chain.invoke({
    "messages" : messages + [HumanMessage(content="What ice cream do I Like")],
    "language" : "English" 
})


response.content


'You like vanilla ice cream.'

In [136]:
## Lets Wrap this in Message History 

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key ="messages",
)

config = {"configurable":{"session_id" : "chat5"}}

In [137]:
response = with_message_history.invoke(
    {
        "messages"  : messages + [HumanMessage(content="What is my name")],
        "language" : "English",
    },
    config = config,
)
response.content

'Your name is Bob.'